In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

os.environ['CUDA_LAUNCH_BLOCKING'] = "0"

import time
import re


import torch as T



import torch.nn as nn
import torch.nn.functional as F


from model import Model


from data_util import config, data
from data_util.batcher import Batcher
from data_util.data import Vocab


from train_util import *
from torch.distributions import Categorical
from rouge import Rouge
from numpy import random
import argparse
import torchsnooper

In [2]:
# random.seed(123)
# T.manual_seed(123)
# if T.cuda.is_available():
#     T.cuda.manual_seed_all(123)
#     print("is_available", T.cuda.is_available())
#     print("use_cuda", use_cuda)

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# T.cuda.set_device(1)

# use_cuda = config.use_gpu and torch.cuda.is_available()
# print("use_cuda", T.cuda.is_available())

# print("use_cuda", use_cuda)

# View batch data

In [3]:
def test_batch():
    vocab = Vocab(config.vocab_path, config.vocab_size)
    batcher = Batcher(config.train_data_path, vocab, mode='train',
                           batch_size=config.batch_size, single_pass=False)
    batch = batcher.next_batch()
    # with torchsnooper.snoop():
    while batch is not None:
        example_list = batch.example_list
        for ex in example_list:
            r = str(ex.original_review)
            s = str(ex.original_summary)
            k = str(ex.key_words)
            sent = ex.original_summary_sents
#             print("original_review_sents:", r)
#             print("original_summary_sents : ", s)
#             print("review_key_words : ", k)
#             print('------------------------------------------------------------\n')
        batch = batcher.next_batch()        
# test_batch()

# Get Bin Information

In [4]:
with open("bin/bin-info.txt",'r',encoding='utf-8') as f:
    lines = f.readlines()
    [print(line) for line in lines]
    train_num = int(lines[0].split(":")[1])
    test_num = int(lines[1].split(":")[1])
    val_num = int(lines[2].split(":")[1])
    # f.write("train : %s\n"%(len(flit_key_train_df)))
    # f.write("test : %s\n"%(len(flit_key_test_df)))
    # f.write("valid : %s\n"%(len(flit_key_valid_df)))


train : 26716

test : 8905

valid : 8905



# View model summary
#### 只有torchsummaryX成功
#### 日後將以此模擬呈現結構

In [ ]:
# from torchsummary import summary # 不支援RNN
# from model import Encoder,Model
# # https://www.cnblogs.com/lindaxin/p/8052043.html
# device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
# encoder = Encoder().to(device)    

# vocab = Vocab(config.vocab_path, config.vocab_size)
# batcher = Batcher(config.train_data_path, vocab, mode='train',
#                        batch_size=config.batch_size, single_pass=False)
# batch = batcher.next_batch()
# enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
# enc_batch = Model().embeds(enc_batch) # Get embeddings for encoder input

# # summary(encoder, enc_batch, enc_lens, show_hierarchical=True) 
# # summary(encoder, [enc_batch, enc_lens])

In [ ]:
# from modelsummary import summary # 未知問題
# from model import Encoder,Model
# # https://www.cnblogs.com/lindaxin/p/8052043.html
# device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
# encoder = Encoder().to(device)    

# vocab = Vocab(config.vocab_path, config.vocab_size)
# batcher = Batcher(config.train_data_path, vocab, mode='train',
#                        batch_size=config.batch_size, single_pass=False)
# batch = batcher.next_batch()
# enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
# enc_batch = Model().embeds(enc_batch) # Get embeddings for encoder input

# # summary(encoder, enc_batch, enc_lens, show_hierarchical=True) 
# # summary(encoder, enc_batch, enc_lens, show_input=False)

In [ ]:
from torchsummaryX import summary
from model import Encoder,Model
device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
encoder = Encoder().to(device)    

vocab = Vocab(config.vocab_path, config.vocab_size)
batcher = Batcher(config.train_data_path, vocab, mode='train',
                       batch_size=config.batch_size, single_pass=False)
batch = batcher.next_batch()
enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
enc_batch = Model(False,'word2Vec',vocab).embeds(enc_batch) #Get embeddings for encoder input

summary(encoder, enc_batch, enc_lens) # encoder summary

           Kernel Shape Output Shape   Params  Mult-Adds
Layer                                                   
0_lstm                -  [690, 1024]  3334144    3325952
1_reduce_h  [1024, 512]     [8, 512]   524800     524288
2_reduce_c  [1024, 512]     [8, 512]   524800     524288
--------------------------------------------------------
                       Totals
Total params          4383744
Trainable params      4383744
Non-trainable params        0
Mult-Adds             4374528


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[690, 1024]",3334144,3325952
1_reduce_h,"[1024, 512]","[8, 512]",524800,524288
2_reduce_c,"[1024, 512]","[8, 512]",524800,524288


# Train object

In [ ]:
# from torchsummaryX import summary
class Train(object):
    def __init__(self, opt):
        self.vocab = Vocab(config.vocab_path, config.vocab_size)
#         print('use %s vocab_size %s '%(opt.word_emb_type,config.vocab_size))

        self.batcher = Batcher(config.train_data_path, self.vocab, mode='train',
                               batch_size=config.batch_size, single_pass=False)
        self.opt = opt
        self.start_id = self.vocab.word2id(data.START_DECODING)
        self.end_id = self.vocab.word2id(data.STOP_DECODING)
        self.pad_id = self.vocab.word2id(data.PAD_TOKEN)
        self.unk_id = self.vocab.word2id(data.UNKNOWN_TOKEN)
        time.sleep(5)

    def save_model(self, iter):
        if not os.path.exists(config.save_model_path):
            os.makedirs(config.save_model_path)
        save_path = config.save_model_path + "/%07d.tar" % iter
        T.save({
            "iter": iter + 1,
            "model_dict": self.model.state_dict(),
            "trainer_dict": self.trainer.state_dict()
        }, save_path)

    def setup_train(self):
        self.model = Model(opt.pre_train_emb,opt.word_emb_type,self.vocab) 
#         print("Model : ",self.model)
#         print("Encoder : ",self.model.encoder)
#         print("Decoder : ",self.model.decoder)
#         print("Embeds : ",self.model.embeds)
        self.model = get_cuda(self.model)
        device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
        if opt.multi_device:
            if T.cuda.device_count() > 1:
                print("Let's use", T.cuda.device_count(), "GPUs!")
                self.model = nn.DataParallel(self.model, list(range(T.cuda.device_count()))).cuda()    
                
                
        if isinstance(self.model,nn.DataParallel):
            self.model = self.model.module
        self.model.to(device)
#         self.model.eval()
        
        self.trainer = T.optim.Adam(self.model.parameters(), lr=config.lr)
        start_iter = 0
        if self.opt.load_model is not None:
            load_model_path = os.path.join(config.save_model_path, self.opt.load_model)
            checkpoint = T.load(load_model_path)
            start_iter = checkpoint["iter"]
            self.model.load_state_dict(checkpoint["model_dict"])
            self.trainer.load_state_dict(checkpoint["trainer_dict"])
            print("Loaded model at " + load_model_path)
        if self.opt.new_lr is not None:
            self.trainer = T.optim.Adam(self.model.parameters(), lr=self.opt.new_lr)
        return start_iter

    def train_batch_MLE(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, batch):
        ''' Calculate Negative Log Likelihood Loss for the given batch. In order to reduce exposure bias,
                pass the previous generated token as input with a probability of 0.25 instead of ground truth label
        Args:
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param batch: batch object
        '''
        dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
        step_losses = []
        s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))                             #Input to the decoder
        prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None                                                                    #Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        for t in range(min(max_dec_len, config.max_dec_steps)):
            use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
            x_t = use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities
            x_t = self.model.embeds(x_t)
            final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
            target = target_batch[:, t]
            log_probs = T.log(final_dist + config.eps)
            step_loss = F.nll_loss(log_probs, target, reduction="none", ignore_index=self.pad_id)
            step_losses.append(step_loss)
            x_t = T.multinomial(final_dist, 1).squeeze()                                            #Sample words from final distribution which can be used as input in next time step
            is_oov = (x_t >= config.vocab_size).long()                                              #Mask indicating whether sampled word is OOV
            x_t = (1 - is_oov) * x_t.detach() + (is_oov) * self.unk_id                              #Replace OOVs with [UNK] token

        losses = T.sum(T.stack(step_losses, 1), 1)                                                  #unnormalized losses for each example in the batch; (batch_size)
        batch_avg_loss = losses / dec_lens                                                          #Normalized losses; (batch_size)
        mle_loss = T.mean(batch_avg_loss)                                                           #Average batch loss
        return mle_loss

    def train_batch_RL(self, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, article_oovs, greedy):
        '''Generate sentences from decoder entirely using sampled tokens as input. These sentences are used for ROUGE evaluation
        Args
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param article_oovs: Batch containing list of OOVs in each example
        :param greedy: If true, performs greedy based sampling, else performs multinomial sampling
        Returns:
        :decoded_strs: List of decoded sentences
        :log_probs: Log probabilities of sampled words
        '''
        s_t = enc_hidden                                                                            #Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))                             #Input to the decoder
        prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None                                                                    #Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        inds = []                                                                                   #Stores sampled indices for each time step
        decoder_padding_mask = []                                                                   #Stores padding masks of generated samples
        log_probs = []                                                                              #Stores log probabilites of generated samples
        mask = get_cuda(T.LongTensor(len(enc_out)).fill_(1))                                        #Values that indicate whether [STOP] token has already been encountered; 1 => Not encountered, 0 otherwise

        for t in range(config.max_dec_steps):
            x_t = self.model.embeds(x_t)
            probs, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
            if greedy is False:
                multi_dist = Categorical(probs)
                x_t = multi_dist.sample()                                                           #perform multinomial sampling
                log_prob = multi_dist.log_prob(x_t)
                log_probs.append(log_prob)
            else:
                _, x_t = T.max(probs, dim=1)                                                        #perform greedy sampling
            x_t = x_t.detach()
            inds.append(x_t)
            mask_t = get_cuda(T.zeros(len(enc_out)))                                                #Padding mask of batch for current time step
            mask_t[mask == 1] = 1                                                                   #If [STOP] is not encountered till previous time step, mask_t = 1 else mask_t = 0
            mask[(mask == 1) + (x_t == self.end_id) == 2] = 0                                       #If [STOP] is not encountered till previous time step and current word is [STOP], make mask = 0
            decoder_padding_mask.append(mask_t)
            is_oov = (x_t>=config.vocab_size).long()                                                #Mask indicating whether sampled word is OOV
            x_t = (1-is_oov)*x_t + (is_oov)*self.unk_id                                             #Replace OOVs with [UNK] token

        inds = T.stack(inds, dim=1)
        decoder_padding_mask = T.stack(decoder_padding_mask, dim=1)
        if greedy is False:                                                                         #If multinomial based sampling, compute log probabilites of sampled words
            log_probs = T.stack(log_probs, dim=1)
            log_probs = log_probs * decoder_padding_mask                                            #Not considering sampled words with padding mask = 0
            lens = T.sum(decoder_padding_mask, dim=1)                                               #Length of sampled sentence
            log_probs = T.sum(log_probs, dim=1) / lens  # (bs,)                                     #compute normalizied log probability of a sentence
        decoded_strs = []
        for i in range(len(enc_out)):
            id_list = inds[i].cpu().numpy()
            oovs = article_oovs[i]
            S = data.outputids2words(id_list, self.vocab, oovs)                                     #Generate sentence corresponding to sampled words
            try:
                end_idx = S.index(data.STOP_DECODING)
                S = S[:end_idx]
            except ValueError:
                S = S
            if len(S) < 2:                                                                           #If length of sentence is less than 2 words, replace it with "xxx"; Avoids setences like "." which throws error while calculating ROUGE
                S = ["xxx"]
            S = " ".join(S)
            decoded_strs.append(S)

        return decoded_strs, log_probs
    
    def train_batch_decode(self,batch, enc_out, enc_hidden, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, article_oovs, greedy):
        '''Generate sentences from decoder entirely using sampled tokens as input. These sentences are used for ROUGE evaluation
        Args
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param article_oovs: Batch containing list of OOVs in each example
        :param greedy: If true, performs greedy based sampling, else performs multinomial sampling
        Returns:
        :decoded_strs: List of decoded sentences
        :log_probs: Log probabilities of sampled words
        '''
        s_t = enc_hidden                                                                            #Decoder hidden states
        x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(self.start_id))                             #Input to the decoder
        prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None                                                                    #Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        inds = []                                                                                   #Stores sampled indices for each time step
        decoder_padding_mask = []                                                                   #Stores padding masks of generated samples
        log_probs = []                                                                              #Stores log probabilites of generated samples
        mask = get_cuda(T.LongTensor(len(enc_out)).fill_(1))                                        #Values that indicate whether [STOP] token has already been encountered; 1 => Not encountered, 0 otherwise

        for t in range(config.max_dec_steps):
            x_t = self.model.embeds(x_t)
#             print('x_t')
#             print(x_t)
            probs, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
            if greedy is False:
                multi_dist = Categorical(probs)
                x_t = multi_dist.sample()                                                           #perform multinomial sampling
#                 log_prob = multi_dist.log_prob(x_t)
#                 log_probs.append(log_prob)
            else:
                _, x_t = T.max(probs, dim=1)                                                        #perform greedy sampling
            x_t = x_t.detach()
            inds.append(x_t)
            mask_t = get_cuda(T.zeros(len(enc_out)))                                                #Padding mask of batch for current time step
            mask_t[mask == 1] = 1                                                                   #If [STOP] is not encountered till previous time step, mask_t = 1 else mask_t = 0
#             mask[(mask == 1) + (x_t == self.end_id) == 2] = 0                                       #If [STOP] is not encountered till previous time step and current word is [STOP], make mask = 0
            decoder_padding_mask.append(mask_t)
#             is_oov = (x_t>=config.vocab_size).long()                                                #Mask indicating whether sampled word is OOV
#             x_t = (1-is_oov)*x_t + (is_oov)*self.unk_id                                             #Replace OOVs with [UNK] token

        inds = T.stack(inds, dim=1)
        decoder_padding_mask = T.stack(decoder_padding_mask, dim=1)
#         if greedy is False:                                                                         #If multinomial based sampling, compute log probabilites of sampled words
#             log_probs = T.stack(log_probs, dim=1)
#             log_probs = log_probs * decoder_padding_mask                                            #Not considering sampled words with padding mask = 0
#             lens = T.sum(decoder_padding_mask, dim=1)                                               #Length of sampled sentence
#             log_probs = T.sum(log_probs, dim=1) / lens  # (bs,)                                     #compute normalizied log probability of a sentence
        decoded_strs = []
        ans_list = []
        for i in range(len(enc_out)):
            id_list = inds[i].cpu().numpy()
            oovs = article_oovs[i]
            S = data.outputids2words(id_list, self.vocab, oovs)                                     #Generate sentence corresponding to sampled words
            try:
                end_idx = S.index(data.STOP_DECODING)
                S = S[:end_idx]
            except ValueError:
                S = S
            if len(S) < 2:                                                                           #If length of sentence is less than 2 words, replace it with "xxx"; Avoids setences like "." which throws error while calculating ROUGE
                S = ["xxx"]
            S = " ".join(S)
            decoded_strs.append(S)
            ans_dict = {
                'review':batch.original_reviews[i],
                'key_words':batch.key_words[i],
                'summary':batch.original_summarys[i],                
                'decoded_str':S                
            }
            ans_list.append(ans_dict)
        return decoded_strs , ans_list

    def reward_function(self, decoded_sents, original_sents):
        rouge = Rouge()
        try:
            scores = rouge.get_scores(decoded_sents, original_sents)
        except Exception:
            print("Rouge failed for multi sentence evaluation.. Finding exact pair")
            scores = []
            for i in range(len(decoded_sents)):
                try:
                    score = rouge.get_scores(decoded_sents[i], original_sents[i])
                except Exception:
                    print("Error occured at:")
                    print("decoded_sents:", decoded_sents[i])
                    print("original_sents:", original_sents[i])
                    score = [{"rouge-l":{"f":0.0}}]
                scores.append(score[0])
        rouge_l_f1 = [score["rouge-l"]["f"] for score in scores]
        rouge_l_f1 = get_cuda(T.FloatTensor(rouge_l_f1))
        return rouge_l_f1 , scores

    # def write_to_file(self, decoded, max, original, sample_r, baseline_r, iter):
    #     with open("temp.txt", "w") as f:
    #         f.write("iter:"+str(iter)+"\n")
    #         for i in range(len(original)):
    #             f.write("dec: "+decoded[i]+"\n")
    #             f.write("max: "+max[i]+"\n")
    #             f.write("org: "+original[i]+"\n")
    #             f.write("Sample_R: %.4f, Baseline_R: %.4f\n\n"%(sample_r[i].item(), baseline_r[i].item()))


    def train_one_batch(self, batch, iter):
        ans_list, batch_scores = None , None
        enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)

        enc_batch = self.model.embeds(enc_batch)                                                    #Get embeddings for encoder input
        enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

        # -------------------------------Summarization-----------------------
        if self.opt.train_mle == True:                                                             #perform MLE training
            mle_loss = self.train_batch_MLE(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch)
        else:
            mle_loss = get_cuda(T.FloatTensor([0]))
            
        if opt.view:
            sample_sents,ans_list = self.train_batch_decode(batch, enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch.art_oovs, greedy=True)
            _ , batch_scores = self.reward_function(sample_sents, batch.original_summarys)
        # --------------RL training-----------------------------------------------------
        if self.opt.train_rl == True:                                                              #perform reinforcement learning training
            # multinomial sampling
            sample_sents, RL_log_probs = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch.art_oovs, greedy=False)
            with T.autograd.no_grad():
                # greedy sampling
                greedy_sents, _ = self.train_batch_RL(enc_out, enc_hidden, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, batch.art_oovs, greedy=True)

            sample_reward , _ = self.reward_function(sample_sents, batch.original_summarys)
            baseline_reward , _ = self.reward_function(greedy_sents, batch.original_summarys)
            # if iter%200 == 0:
            #     self.write_to_file(sample_sents, greedy_sents, batch.original_abstracts, sample_reward, baseline_reward, iter)
            rl_loss = -(sample_reward - baseline_reward) * RL_log_probs                             #Self-critic policy gradient training (eq 15 in https://arxiv.org/pdf/1705.04304.pdf)
            rl_loss = T.mean(rl_loss)

            batch_reward = T.mean(sample_reward).item()
        else:
            rl_loss = get_cuda(T.FloatTensor([0]))
            batch_reward = 0

    # ------------------------------------------------------------------------------------
#         if opt.train_mle == True: 
        self.trainer.zero_grad()
        (self.opt.mle_weight * mle_loss + self.opt.rl_weight * rl_loss).backward()
        self.trainer.step()       
        
        return mle_loss.item(), batch_reward, ans_list, batch_scores
        
        
    def get_best_res_score(self,results,scores):
        max_score = float(0)
        _id = 0
        for idx in range(len(results)):       
            re_matchData = re.compile(r'\-?\d{1,10}\.?\d{1,10}')
            data = re.findall(re_matchData, str(scores[idx]))
            score = sum([float(d) for d in data])
            if score > max_score: 
                _id = idx
        return results[_id],scores[_id]
            
        
    
    def trainIters(self):
        iter = self.setup_train()
        epoch = 0
        count = mle_total = r_total = 0
        print("Start training.....")
        while iter <= config.max_iterations:
#         while iter <= config.max_epochs:
            f = open('training_log.txt','a+',encoding='utf-8')
            batch = self.batcher.next_batch()
            try:
                mle_loss, r , ans_list, batch_scores = self.train_one_batch(batch, iter)
#                 print(iter,mle_loss)
#                 break
            except KeyboardInterrupt:
                print("-------------------Keyboard Interrupt------------------")
                exit(0)
#             if opt.train_mle == False: break
            mle_total += mle_loss
            r_total += r
            count += 1
            iter += 1

            if iter % 1000 == 0:
                mle_avg = mle_total / count
                r_avg = r_total / count
                epoch = int((iter * config.batch_size) / train_num) + 1
                print('epoch:',epoch ,"iter:", iter, "mle_loss:", "%.3f" % mle_avg, "reward:", "%.4f" % r_avg)
                f.write('epoch: %s iter: %s mle_loss: %.3f reward: %.3f \n'%(epoch,iter,mle_avg,r_avg))
                if opt.view :
                    best_res , best_score = self.get_best_res_score(ans_list,batch_scores)
                    print('best_res:',best_res); f.write('best_res: %s \n' %(best_res))
                    print('best_score:',best_score); f.write('best_score: %s \n' %(best_score))
                count = mle_total = r_total = 0
                f.close()
#                 break

            if iter % 5000 == 0:
                self.save_model(iter)

In [ ]:
# https://blog.csdn.net/u012869752/article/details/72513141
# 由于在jupyter notebook中，args不为空

# nvidia-smi -pm 1

if __name__ == "__main__":   
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_mle', type=bool, default=True)
    parser.add_argument('--train_rl', type=bool, default=False)
    parser.add_argument('--mle_weight', type=float, default=1.0)
    parser.add_argument('--load_model', type=str, default=None)
    parser.add_argument('--new_lr', type=float, default=None)
    parser.add_argument('--multi_device', type=bool, default=True)
    parser.add_argument('--view', type=bool, default=True)
    parser.add_argument('--pre_train_emb', type=bool, default=True)
    parser.add_argument('--word_emb_type', type=str, default='glove')
    parser.add_argument('--truncate', type=bool, default=False)
    opt = parser.parse_args(args=[])

    opt.rl_weight = 1 - opt.mle_weight  
    
#     config.vocab_path = "Embedding/%s/%s.vocab"%(opt.word_emb_type,opt.word_emb_type)
#     if config.word_emb_type == "glove": 
#         config.vocab_size = len(open(config.vocab_path, 'r',encoding='utf-8').readlines())
    
    if opt.truncate :
        if os.path.exists('training_log.txt'):
            with open('training_log.txt','r+',encoding='utf-8') as q :
                q.truncate()     
        else:
            open('training_log.txt','w').close()                

    f = open('training_log.txt','a+',encoding='utf-8')
    if opt.train_rl == True:
        print("Training mle: %s, Training rl: %s, mle weight: %.2f, rl weight: %.2f"%(opt.train_mle, opt.train_rl, opt.mle_weight, opt.rl_weight))
        f.write("Training mle: %s, Training rl: %s, mle weight: %.2f, rl weight: %.2f \n"%(opt.train_mle, opt.train_rl, opt.mle_weight, opt.rl_weight))
    else:
        print("Training mle: %s, mle weight: %.2f"%(opt.train_mle, opt.mle_weight))
        f.write("Training mle: %s, mle weight: %.2f \n"%(opt.train_mle, opt.mle_weight))

    if opt.pre_train_emb :
#         with open(config.vocab_path,'r',encoding='utf-8') as q :
#             config.vocab_size = len(q.readlines())   
        print('use pre_train_%s vocab_size %s '%(opt.word_emb_type,config.vocab_size))
        f.write('use pre_train_%s vocab_size %s \n'%(opt.word_emb_type,config.vocab_size))
        
    else:
        print('use %s %s vocab_size %s '%(opt.word_emb_type,config.vocab_size))
        f.write('use %s %s vocab_size %s \n'%(opt.word_emb_type,config.vocab_size))

    print("intra_encoder:", config.intra_encoder, "intra_decoder:", config.intra_decoder)
    f.write("intra_encoder: %s intra_decoder: %s \n"%(config.intra_encoder, config.intra_decoder))
    f.close()
    
    with open('training_log.txt','a+',encoding='utf-8') as f :
        f.write('----------------------------------------------------------------\n')
    train_processor = Train(opt)
    train_processor.trainIters()

Training mle: True, mle weight: 1.00
use pre_train_glove vocab_size 50000 
intra_encoder: True intra_decoder: True
Let's use 2 GPUs!
Start training.....
epoch: 1 iter: 1000 mle_loss: 3.643 reward: 0.0000
best_res: {'review': 'this small case but fit both nikon coolpix and husband olympus snugly without too tight water resistant which great for when to the beach waterpark and nicely pad inside to help protect the camera the color bright and vibrant make easier to see at the bottom beach bag purse can just toss and have put the camera strap to keep from catch the zipper and so far work great ', 'key_words': ['beach', 'protect', 'camera', 'bottom', 'purse'], 'summary': '<s> cute little case </s>', 'decoded_str': '[UNK] great case [UNK]'}
best_score: {'rouge-1': {'f': 0.24999999531250006, 'p': 0.3333333333333333, 'r': 0.2}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.22368421052631188, 'p': 0.3333333333333333, 'r': 0.2}}
epoch: 1 iter: 2000 mle_loss: 3.539 reward: 0.0000


In [ ]:
# if self.opt.train_rl == "yes":
# ~/Users/leyan/Text-Summarizer-FOP/data_util/data.py in outputids2words(id_list, vocab, review_oovs)
#     141       review_oov_idx = i - vocab.size()
#     142       try:
# --> 143         w = review_oovs[review_oov_idx]
#     144       except ValueError as e: # i doesn't correspond to an review oov
#     145         raise ValueError('Error: model produced word ID %i which corresponds to review OOV %i but this example only has %i review OOVs' % (i, review_oov_idx, len(review_oovs)))

# IndexError: list index out of range